Pandas user-defined function (UDF): also known as vectorized UDF

It is a user-defined function that uses Apache Arrow to transfer data and pandas to work with the data

Pandas UDFs allow vectorized operations that can increase performance up to 100x compared to row-at-a-time Python UDFs

## Type hint in pandas udf

- In Python, type hints are used to statically indicate the type of a value. This can be done for variables, parameters, function arguments, and return values
- In the context of Pandas UDFs, type hints are used to specify the types of the input and output arguments of the UDF
- This can help to improve the the readability, maintainability of the code, and helps to catch errors at compile time

In [0]:
import pandas as pd

In [0]:
def multiply(a: pd.Series, b: pd.Series) -> pd.Series:
    return a*b

In [0]:

x = pd.Series([2, 3, 4])
y = pd.Series([5, 6, 7])

In [0]:
print(multiply(x, y))

0    10
1    18
2    28
dtype: int64


# 1. Series to Series UDF

- Use: To vectorize scalar operations & with APIs such as select and withColumn
- Spark runs a pandas UDF by splitting columns into batches, calling the function for each batch as a subset of the data, then concatenating the results

In [0]:
# Import necessary libraries
import pandas as pd
from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import LongType

In [0]:
# Declare the function and create the UDF
def multiply_func(a: pd.Series, b: pd.Series) -> pd.Series:
    return a * b
 
multiply = pandas_udf(multiply_func, returnType=LongType())

In [0]:
# Test the Function Locally
x = pd.Series([1, 2, 3])
print(multiply_func(x, x))

0    1
1    4
2    9
dtype: int64


In [0]:
# Create a Spark DataFrame
df = spark.createDataFrame(pd.DataFrame(x, columns=["x"]))

In [0]:
df.show()

+---+
|  x|
+---+
|  1|
|  2|
|  3|
+---+



In [0]:
# Execute function as a Spark vectorized UDF
df.select(multiply(col("x"), col("x"))).show()

+-------------------+
|multiply_func(x, x)|
+-------------------+
|                  1|
|                  4|
|                  9|
+-------------------+



# 2. Iterator of Series to Iterator of Series UDF

An iterator UDF is the same as a scalar pandas UDF except:

1. The Python function: Takes an iterator of batches instead of a single input batch as input, Returns an iterator of output batches instead of a single output batch
2. The length of the entire output in the iterator should be the same as the length of the entire input
3. The wrapped pandas UDF takes a single Spark column as an input

- Useful when the UDF execution requires initializing some state
- For example, loading a ML model file to apply inference to every input batch

In [0]:
# Import necessary libraries
import pandas as pd
from typing import Iterator
from pyspark.sql.functions import col, pandas_udf, struct

In [0]:
# Create a Pandas DataFrame
pdf = pd.DataFrame([1, 2, 3], columns=["x"])
 
# Convert the Pandas DataFrame to a Spark DataFrame
df = spark.createDataFrame(pdf)

In [0]:
df.show()

+---+
|  x|
+---+
|  1|
|  2|
|  3|
+---+



In [0]:
# Define a Pandas UDF 'plus_one'
 
# When the UDF is called with the column, the input to the underlying function is an iterator of pd.Series
@pandas_udf("long")
def plus_one(batch_iter: Iterator[pd.Series]) -> Iterator[pd.Series]:
    for x in batch_iter:
        yield x + 1

In [0]:

# Apply the 'plus_one' UDF to the "x" column of the Spark DataFrame
df.select(plus_one(col("x"))).show()

+-----------+
|plus_one(x)|
+-----------+
|          2|
|          3|
|          4|
+-----------+



In [0]:
# Define another Pandas UDF named 'plus_y'
 
y_bc = spark.sparkContext.broadcast(1)
 
@pandas_udf("long")
def plus_y(batch_iter: Iterator[pd.Series]) -> Iterator[pd.Series]:
    y = y_bc.value  # initialize states
    try:
        for x in batch_iter:
            yield x + y
    finally:
        pass  # release resources here, if any

In [0]:

# Apply the 'plus_y' UDF to the "x" column of the Spark DataFrame
df.select(plus_y(col("x"))).show()

+---------+
|plus_y(x)|
+---------+
|        2|
|        3|
|        4|
+---------+



# 3. Iterator of multiple Series to Iterator of Series UDF


- Similar characteristics and restrictions as Iterator of Series to Iterator of Series UDF
- The specified function takes an iterator of batches and outputs an iterator of batches
- It is also useful when the UDF execution requires initializing some state

The differences are:

- The underlying Python function takes an iterator of a tuple of pandas Series
- The wrapped pandas UDF takes multiple Spark columns as an input

In [0]:
# Importing Libraries
import pandas as pd
from typing import Iterator, Tuple
from pyspark.sql.functions import col, pandas_udf, struct

In [0]:

# Create a Pandas DataFrame
pdf = pd.DataFrame([1, 2, 3], columns=["x"])
 
# Convert the Pandas DataFrame to a Spark DataFrame
df = spark.createDataFrame(pdf)

In [0]:
df.show()

+---+
|  x|
+---+
|  1|
|  2|
|  3|
+---+

